# Hodina 15: Řešení CSP a propagace omezení 🔗

## Přehled lekce

V této hodině se naučíme:
- Pokročilé techniky propagace omezení (AC-3, PC)
- Globální omezení a jejich implementace s neuronovými sítěmi
- Symmetry breaking s transformery
- Paralelní CSP solving
- Vytvoření pokročilého CSP řešiče

---

## 1. Nastavení prostředí a instalace knihoven

In [2]:
# Instalace potřebných knihoven
!pip install torch torchvision transformers gradio networkx matplotlib numpy
!pip install plotly python-constraint ortools ray[default] joblib

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import networkx as nx
from collections import defaultdict, deque
import time
from IPython.display import HTML, display, clear_output
import plotly.graph_objects as go
import gradio as gr
from transformers import GPT2Model, GPT2Config
import json
import random
from typing import List, Dict, Set, Tuple, Optional
from constraint import *
from ortools.sat.python import cp_model
import ray
from joblib import Parallel, delayed
import itertools

# Nastavení zobrazení
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
torch.manual_seed(42)

zsh:1: no matches found: ray[default]


ModuleNotFoundError: No module named 'plotly'

## 2. Pokročilé Arc Consistency algoritmy 🏹

Implementujeme AC-3, AC-4 a Path Consistency.

In [3]:
# Pokročilý framework pro propagaci omezení
class AdvancedConstraintPropagation:
    def __init__(self, csp):
        self.csp = csp
        self.revision_count = 0
        self.propagation_history = []
    
    def ac3(self, domains=None):
        """Arc Consistency 3 algoritmus"""
        if domains is None:
            domains = {var: list(self.csp.domains[var]) for var in self.csp.variables}
        
        # Inicializace fronty všemi oblouky
        queue = deque()
        for constraint in self.csp.constraints:
            if len(constraint.variables) == 2:
                v1, v2 = constraint.variables
                queue.append((v1, v2, constraint))
                queue.append((v2, v1, constraint))
        
        while queue:
            xi, xj, constraint = queue.popleft()
            
            if self.revise(xi, xj, constraint, domains):
                self.revision_count += 1
                
                # Uložení kroku pro vizualizaci
                self.propagation_history.append({
                    'type': 'revision',
                    'variables': (xi, xj),
                    'domains': {var: list(domains[var]) for var in self.csp.variables}
                })
                
                if len(domains[xi]) == 0:
                    return False, domains
                
                # Přidání sousedů xi do fronty
                for other_constraint in self.csp.constraints:
                    if xi in other_constraint.variables:
                        for xk in other_constraint.variables:
                            if xk != xi and xk != xj:
                                queue.append((xk, xi, other_constraint))
        
        return True, domains
    
    def revise(self, xi, xj, constraint, domains):
        """Redukuje doménu xi vzhledem k xj"""
        revised = False
        to_remove = []
        
        for x in domains[xi]:
            has_support = False
            
            for y in domains[xj]:
                assignment = {xi: x, xj: y}
                if constraint.is_satisfied(assignment):
                    has_support = True
                    break
            
            if not has_support:
                to_remove.append(x)
                revised = True
        
        for x in to_remove:
            domains[xi].remove(x)
        
        return revised
    
    def ac4(self):
        """AC-4 algoritmus - efektivnější verze AC-3"""
        domains = {var: list(self.csp.domains[var]) for var in self.csp.variables}
        
        # Fáze 1: Inicializace podpor
        support = defaultdict(lambda: defaultdict(set))
        counter = defaultdict(lambda: defaultdict(int))
        
        for constraint in self.csp.constraints:
            if len(constraint.variables) == 2:
                v1, v2 = constraint.variables
                
                for x in domains[v1]:
                    total = 0
                    for y in domains[v2]:
                        if constraint.is_satisfied({v1: x, v2: y}):
                            support[v2][y].add((v1, x))
                            total += 1
                    counter[v1][x] = total
                    
                    if total == 0:
                        domains[v1].remove(x)
        
        # Fáze 2: Propagace
        queue = deque()
        for var in self.csp.variables:
            for val in self.csp.domains[var]:
                if val not in domains[var]:
                    queue.append((var, val))
        
        while queue:
            vj, y = queue.popleft()
            
            for vi, x in support[vj][y]:
                counter[vi][x] -= 1
                
                if counter[vi][x] == 0 and x in domains[vi]:
                    domains[vi].remove(x)
                    queue.append((vi, x))
        
        return all(len(domains[var]) > 0 for var in self.csp.variables), domains
    
    def path_consistency(self):
        """Path Consistency (PC-2) algoritmus"""
        domains = {var: list(self.csp.domains[var]) for var in self.csp.variables}
        
        # Vytvoření binárních omezení mezi všemi páry proměnných
        binary_constraints = defaultdict(list)
        for constraint in self.csp.constraints:
            if len(constraint.variables) == 2:
                v1, v2 = constraint.variables
                binary_constraints[(v1, v2)].append(constraint)
                binary_constraints[(v2, v1)].append(constraint)
        
        changed = True
        while changed:
            changed = False
            
            # Pro každou trojici proměnných
            for vi, vj, vk in itertools.permutations(self.csp.variables, 3):
                # Kontrola path consistency
                to_remove = []
                
                for x in domains[vi]:
                    for y in domains[vj]:
                        has_path = False
                        
                        for z in domains[vk]:
                            # Kontrola, zda existuje konzistentní cesta
                            path_valid = True
                            
                            # Kontrola vi-vk
                            if (vi, vk) in binary_constraints:
                                for c in binary_constraints[(vi, vk)]:
                                    if not c.is_satisfied({vi: x, vk: z}):
                                        path_valid = False
                                        break
                            
                            # Kontrola vk-vj
                            if path_valid and (vk, vj) in binary_constraints:
                                for c in binary_constraints[(vk, vj)]:
                                    if not c.is_satisfied({vk: z, vj: y}):
                                        path_valid = False
                                        break
                            
                            if path_valid:
                                has_path = True
                                break
                        
                        if not has_path:
                            to_remove.append((x, y))
                            changed = True
                
                # Odstranění nekonzistentních párů
                # (V této implementaci jen zaznamenáme)
                if to_remove:
                    self.propagation_history.append({
                        'type': 'path_revision',
                        'variables': (vi, vj, vk),
                        'removed_pairs': to_remove
                    })
        
        return True, domains
    
    def visualize_propagation(self):
        """Vizualizuje průběh propagace"""
        if not self.propagation_history:
            print("Žádná historie propagace")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.flatten()
        
        # Vybrat 4 klíčové kroky
        steps = [0, len(self.propagation_history)//3, 
                2*len(self.propagation_history)//3, -1]
        
        for idx, step_idx in enumerate(steps):
            if step_idx >= len(self.propagation_history):
                continue
            
            step = self.propagation_history[step_idx]
            ax = axes[idx]
            
            # Vizualizace domén
            domains = step['domains']
            vars_list = list(domains.keys())
            n_vars = len(vars_list)
            
            # Matice velikostí domén
            domain_sizes = [len(domains[var]) for var in vars_list]
            
            im = ax.bar(range(n_vars), domain_sizes, color='skyblue')
            ax.set_xticks(range(n_vars))
            ax.set_xticklabels(vars_list, rotation=45)
            ax.set_ylabel('Velikost domény')
            ax.set_title(f'Krok {step_idx + 1}')
            
            # Zvýraznění revidovaných proměnných
            if step['type'] == 'revision':
                v1, v2 = step['variables']
                if v1 in vars_list:
                    idx1 = vars_list.index(v1)
                    im[idx1].set_color('orange')
                if v2 in vars_list:
                    idx2 = vars_list.index(v2)
                    im[idx2].set_color('red')
        
        plt.tight_layout()
        plt.show()

# Demonstrace propagace omezení
print("🏹 Demonstrace pokročilé propagace omezení:")

# Vytvoření testovacího problému
from Hour_14_content import CSPProblem, BinaryConstraint  # Import z předchozí hodiny

# Jednoduchý testovací problém
test_csp = CSPProblem()
test_csp.add_variable('A', [1, 2, 3, 4])
test_csp.add_variable('B', [2, 3, 4, 5])
test_csp.add_variable('C', [1, 3, 5, 7])

# Omezení: A < B, B != C
test_csp.add_constraint(BinaryConstraint('A', 'B', lambda a, b: a < b))
test_csp.add_constraint(BinaryConstraint('B', 'C', lambda b, c: b != c))

# Test AC-3
propagator = AdvancedConstraintPropagation(test_csp)
consistent, reduced_domains = propagator.ac3()

print(f"AC-3 výsledek: {'Konzistentní' if consistent else 'Nekonzistentní'}")
print(f"Počet revizí: {propagator.revision_count}")
print("\nRedukované domény:")
for var, domain in reduced_domains.items():
    print(f"{var}: {domain}")

# Vizualizace
propagator.visualize_propagation()

🏹 Demonstrace pokročilé propagace omezení:


ModuleNotFoundError: No module named 'Hour_14_content'

## 3. Globální omezení s neuronovými sítěmi 🌐

Implementujeme pokročilá globální omezení a jejich neuronové reprezentace.

In [4]:
# Globální omezení
class GlobalConstraint:
    """Základní třída pro globální omezení"""
    def __init__(self, variables):
        self.variables = variables
    
    def is_satisfied(self, assignment):
        raise NotImplementedError
    
    def propagate(self, domains):
        """Propaguje omezení a redukuje domény"""
        raise NotImplementedError

class AllDifferentGlobal(GlobalConstraint):
    """Globální AllDifferent s efektivní propagací"""
    def is_satisfied(self, assignment):
        values = [assignment[var] for var in self.variables if var in assignment]
        return len(values) == len(set(values))
    
    def propagate(self, domains):
        """Hall's theorem based propagation"""
        # Vytvoření bipartitního grafu
        G = nx.Graph()
        
        # Přidání uzlů
        G.add_nodes_from(self.variables, bipartite=0)
        all_values = set()
        for var in self.variables:
            all_values.update(domains[var])
        G.add_nodes_from(all_values, bipartite=1)
        
        # Přidání hran
        for var in self.variables:
            for val in domains[var]:
                G.add_edge(var, val)
        
        # Nalezení maximálního párování
        matching = nx.bipartite.maximum_matching(G, self.variables)
        
        if len(matching) < len(self.variables) * 2:
            return False, domains  # Nekonzistentní
        
        # TODO: Implementovat úplnou Hall's theorem propagaci
        return True, domains

class SumConstraint(GlobalConstraint):
    """Omezení na součet proměnných"""
    def __init__(self, variables, target_sum):
        super().__init__(variables)
        self.target_sum = target_sum
    
    def is_satisfied(self, assignment):
        if all(var in assignment for var in self.variables):
            return sum(assignment[var] for var in self.variables) == self.target_sum
        return True
    
    def propagate(self, domains):
        """Bounds consistency propagace"""
        # Výpočet minimálního a maximálního možného součtu
        min_sum = sum(min(domains[var]) for var in self.variables)
        max_sum = sum(max(domains[var]) for var in self.variables)
        
        if min_sum > self.target_sum or max_sum < self.target_sum:
            return False, domains
        
        # Redukce domén na základě bounds
        for var in self.variables:
            other_vars = [v for v in self.variables if v != var]
            
            # Minimální hodnota pro var
            max_others = sum(max(domains[v]) for v in other_vars)
            min_var = self.target_sum - max_others
            
            # Maximální hodnota pro var
            min_others = sum(min(domains[v]) for v in other_vars)
            max_var = self.target_sum - min_others
            
            # Redukce domény
            domains[var] = [v for v in domains[var] if min_var <= v <= max_var]
            
            if not domains[var]:
                return False, domains
        
        return True, domains

# Neuronová síť pro učení globálních omezení
class NeuralConstraintLearner(nn.Module):
    def __init__(self, max_vars=20, max_domain=50, hidden_dim=256):
        super(NeuralConstraintLearner, self).__init__()
        
        # Encoder pro proměnné a jejich domény
        self.var_encoder = nn.LSTM(
            input_size=max_domain,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            bidirectional=True
        )
        
        # Attention mechanismus
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim * 2,
            num_heads=8,
            batch_first=True
        )
        
        # Klasifikátor omezení
        self.constraint_classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, 5)  # 5 typů omezení
        )
        
        # Prediktor parametrů omezení
        self.parameter_predictor = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 10)  # Max 10 parametrů
        )
    
    def forward(self, domain_matrix, valid_mask):
        # domain_matrix: [batch, n_vars, max_domain]
        # valid_mask: [batch, n_vars]
        
        # Encode proměnné
        encoded, _ = self.var_encoder(domain_matrix)
        
        # Self-attention
        attended, _ = self.attention(encoded, encoded, encoded,
                                     key_padding_mask=~valid_mask)
        
        # Global pooling
        pooled = attended.mean(dim=1)
        
        # Predikce typu omezení a parametrů
        constraint_type = self.constraint_classifier(pooled)
        parameters = self.parameter_predictor(pooled)
        
        return constraint_type, parameters

# Demonstrace globálních omezení
print("\n🌐 Demonstrace globálních omezení:")

# Vytvoření problému s globálními omezeními
class GlobalCSPProblem(CSPProblem):
    def __init__(self):
        super().__init__()
        self.global_constraints = []
    
    def add_global_constraint(self, constraint):
        self.global_constraints.append(constraint)
    
    def propagate_global_constraints(self, domains):
        """Propaguje všechna globální omezení"""
        for constraint in self.global_constraints:
            consistent, domains = constraint.propagate(domains)
            if not consistent:
                return False, domains
        return True, domains

# Příklad: Magic Square
def create_magic_square_problem(n=3):
    """Vytvoří problém magického čtverce"""
    problem = GlobalCSPProblem()
    magic_sum = n * (n*n + 1) // 2
    
    # Proměnné
    for i in range(n):
        for j in range(n):
            problem.add_variable(f'cell_{i}_{j}', list(range(1, n*n + 1)))
    
    # Globální AllDifferent
    all_vars = [f'cell_{i}_{j}' for i in range(n) for j in range(n)]
    problem.add_global_constraint(AllDifferentGlobal(all_vars))
    
    # Omezení součtů pro řádky
    for i in range(n):
        row_vars = [f'cell_{i}_{j}' for j in range(n)]
        problem.add_global_constraint(SumConstraint(row_vars, magic_sum))
    
    # Omezení součtů pro sloupce
    for j in range(n):
        col_vars = [f'cell_{i}_{j}' for i in range(n)]
        problem.add_global_constraint(SumConstraint(col_vars, magic_sum))
    
    # Diagonály
    diag1 = [f'cell_{i}_{i}' for i in range(n)]
    diag2 = [f'cell_{i}_{n-1-i}' for i in range(n)]
    problem.add_global_constraint(SumConstraint(diag1, magic_sum))
    problem.add_global_constraint(SumConstraint(diag2, magic_sum))
    
    return problem

# Test
magic_square = create_magic_square_problem(3)
print(f"Magický čtverec 3x3 (magický součet = 15)")
print(f"Počet proměnných: {len(magic_square.variables)}")
print(f"Počet globálních omezení: {len(magic_square.global_constraints)}")

# Propagace
initial_domains = {var: list(magic_square.domains[var]) 
                  for var in magic_square.variables}
consistent, reduced = magic_square.propagate_global_constraints(initial_domains)

print(f"\nPo propagaci globálních omezení:")
print(f"Konzistentní: {consistent}")
total_reduction = sum(len(initial_domains[var]) - len(reduced[var]) 
                     for var in magic_square.variables)
print(f"Celková redukce domén: {total_reduction} hodnot")


🌐 Demonstrace globálních omezení:


NameError: name 'CSPProblem' is not defined

## 4. Symmetry Breaking s transformery 🔄

Detekce a eliminace symetrií pro efektivnější řešení.

In [5]:
# Detekce a breaking symetrií
class SymmetryDetector:
    def __init__(self, csp):
        self.csp = csp
        self.symmetries = []
    
    def detect_variable_symmetries(self):
        """Detekuje symetrie mezi proměnnými"""
        var_symmetries = []
        
        for v1, v2 in itertools.combinations(self.csp.variables, 2):
            # Kontrola, zda mají stejné domény
            if set(self.csp.domains[v1]) == set(self.csp.domains[v2]):
                # Kontrola, zda jsou zaměnitelné v omezeních
                if self._are_interchangeable(v1, v2):
                    var_symmetries.append((v1, v2))
        
        return var_symmetries
    
    def _are_interchangeable(self, v1, v2):
        """Kontroluje, zda jsou dvě proměnné zaměnitelné"""
        # Získat omezení obsahující v1 nebo v2
        v1_constraints = [c for c in self.csp.constraints if v1 in c.variables]
        v2_constraints = [c for c in self.csp.constraints if v2 in c.variables]
        
        # Jednoduchá kontrola - mají stejný počet omezení?
        if len(v1_constraints) != len(v2_constraints):
            return False
        
        # TODO: Pokročilejší kontrola strukturální ekvivalence
        return True
    
    def detect_value_symmetries(self):
        """Detekuje symetrie mezi hodnotami"""
        value_symmetries = defaultdict(list)
        
        for var in self.csp.variables:
            domain = self.csp.domains[var]
            
            # Pro každou dvojici hodnot
            for val1, val2 in itertools.combinations(domain, 2):
                # Kontrola, zda jsou zaměnitelné
                if self._values_are_symmetric(var, val1, val2):
                    value_symmetries[var].append((val1, val2))
        
        return value_symmetries
    
    def _values_are_symmetric(self, var, val1, val2):
        """Kontroluje symetrii hodnot"""
        # Jednoduchá implementace - kontrola v binárních omezeních
        for constraint in self.csp.constraints:
            if var in constraint.variables and len(constraint.variables) == 2:
                other_var = [v for v in constraint.variables if v != var][0]
                
                # Kontrola, zda val1 a val2 mají stejné vztahy s ostatními hodnotami
                for other_val in self.csp.domains[other_var]:
                    sat1 = constraint.is_satisfied({var: val1, other_var: other_val})
                    sat2 = constraint.is_satisfied({var: val2, other_var: other_val})
                    if sat1 != sat2:
                        return False
        
        return True
    
    def add_symmetry_breaking_constraints(self):
        """Přidá omezení pro eliminaci symetrií"""
        breaking_constraints = []
        
        # Lexikografické uspořádání pro symetrické proměnné
        var_symmetries = self.detect_variable_symmetries()
        for v1, v2 in var_symmetries:
            # Přidat omezení v1 <= v2
            breaking_constraints.append(
                BinaryConstraint(v1, v2, lambda x, y: x <= y)
            )
        
        return breaking_constraints

# Transformer pro automatickou detekci symetrií
class SymmetryTransformer(nn.Module):
    def __init__(self, d_model=256, n_heads=8, n_layers=4):
        super(SymmetryTransformer, self).__init__()
        
        # Encoder pro constraint graf
        self.graph_encoder = nn.Sequential(
            nn.Linear(100, d_model),  # Předpokládáme max 100 features
            nn.ReLU(),
            nn.LayerNorm(d_model)
        )
        
        # Transformer layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Symetry detector heads
        self.var_symmetry_head = nn.Linear(d_model * 2, 1)
        self.value_symmetry_head = nn.Linear(d_model, 1)
    
    def forward(self, node_features, edge_features, adjacency_matrix):
        # node_features: [batch, n_vars, features]
        # edge_features: [batch, n_vars, n_vars, features]
        # adjacency_matrix: [batch, n_vars, n_vars]
        
        # Encode nodes
        encoded_nodes = self.graph_encoder(node_features)
        
        # Transformer processing s attention maskou
        attention_mask = adjacency_matrix == 0  # Maskovat neexistující hrany
        transformed = self.transformer(encoded_nodes, src_key_padding_mask=attention_mask)
        
        # Detekce symetrií mezi proměnnými
        n_vars = transformed.size(1)
        var_pairs = []
        
        for i in range(n_vars):
            for j in range(i+1, n_vars):
                pair_features = torch.cat([transformed[:, i], transformed[:, j]], dim=-1)
                symmetry_score = self.var_symmetry_head(pair_features).sigmoid()
                var_pairs.append((i, j, symmetry_score))
        
        # Detekce symetrií hodnot
        value_symmetries = self.value_symmetry_head(transformed).sigmoid()
        
        return var_pairs, value_symmetries

# Demonstrace symmetry breaking
print("\n🔄 Demonstrace Symmetry Breaking:")

# Vytvoření symetrického problému - Graph Coloring
def create_symmetric_coloring_problem():
    problem = CSPProblem()
    
    # Kompletní graf K4
    nodes = ['A', 'B', 'C', 'D']
    colors = ['R', 'G', 'B']
    
    # Všechny uzly mají stejnou doménu
    for node in nodes:
        problem.add_variable(node, colors)
    
    # Všechny páry musí mít různé barvy
    for n1, n2 in itertools.combinations(nodes, 2):
        problem.add_constraint(
            BinaryConstraint(n1, n2, lambda x, y: x != y)
        )
    
    return problem

# Test detekce symetrií
sym_problem = create_symmetric_coloring_problem()
detector = SymmetryDetector(sym_problem)

var_syms = detector.detect_variable_symmetries()
val_syms = detector.detect_value_symmetries()

print("Detekované symetrie proměnných:")
for v1, v2 in var_syms:
    print(f"  {v1} ≈ {v2}")

print("\nDetekované symetrie hodnot:")
for var, syms in val_syms.items():
    if syms:
        print(f"  {var}: {syms}")

# Přidání symmetry breaking constraints
breaking_constraints = detector.add_symmetry_breaking_constraints()
print(f"\nPřidáno {len(breaking_constraints)} symmetry breaking omezení")

# Vizualizace symetrií
plt.figure(figsize=(8, 6))
G = nx.complete_graph(4)
pos = nx.spring_layout(G)

# Kreslení grafu
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=1000)
nx.draw_networkx_labels(G, pos, {i: nodes[i] for i in range(4)})
nx.draw_networkx_edges(G, pos, edge_color='gray')

# Zvýraznění symetrických párů
sym_edges = [(nodes.index(v1), nodes.index(v2)) for v1, v2 in var_syms]
nx.draw_networkx_edges(G, pos, edgelist=sym_edges, 
                      edge_color='red', width=3, style='dashed')

plt.title('Graf s detekovanými symetriemi (červené čáry)')
plt.axis('off')
plt.show()


🔄 Demonstrace Symmetry Breaking:


NameError: name 'CSPProblem' is not defined

## 5. Paralelní CSP solving 🚀

Využití paralelizace pro rychlejší řešení CSP.

In [6]:
# Paralelní CSP solver
import ray
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp

# Inicializace Ray (pokud ještě není)
if not ray.is_initialized():
    ray.init(ignore_reinit_error=True)

@ray.remote
class ParallelCSPWorker:
    """Worker pro paralelní řešení CSP"""
    def __init__(self, worker_id):
        self.worker_id = worker_id
        self.nodes_explored = 0
    
    def solve_subproblem(self, csp, initial_assignment, variable_order):
        """Řeší podproblém s daným počátečním přiřazením"""
        return self._backtrack(csp, initial_assignment, variable_order)
    
    def _backtrack(self, csp, assignment, variable_order):
        if len(assignment) == len(csp.variables):
            return assignment
        
        # Výběr další proměnné
        unassigned = [v for v in variable_order if v not in assignment]
        if not unassigned:
            return None
        
        var = unassigned[0]
        
        for value in csp.domains[var]:
            self.nodes_explored += 1
            
            # Kontrola konzistence
            assignment[var] = value
            consistent = all(
                constraint.is_satisfied(assignment)
                for constraint in csp.constraints
                if all(v in assignment for v in constraint.variables)
            )
            
            if consistent:
                result = self._backtrack(csp, assignment.copy(), variable_order)
                if result is not None:
                    return result
            
            del assignment[var]
        
        return None
    
    def get_stats(self):
        return {
            'worker_id': self.worker_id,
            'nodes_explored': self.nodes_explored
        }

class ParallelCSPSolver:
    def __init__(self, n_workers=4):
        self.n_workers = n_workers
        self.workers = [ParallelCSPWorker.remote(i) for i in range(n_workers)]
    
    def solve(self, csp, strategy='domain_split'):
        """Paralelně řeší CSP"""
        if strategy == 'domain_split':
            return self._solve_domain_split(csp)
        elif strategy == 'variable_split':
            return self._solve_variable_split(csp)
        else:
            raise ValueError(f"Unknown strategy: {strategy}")
    
    def _solve_domain_split(self, csp):
        """Rozdělí doménu první proměnné mezi workery"""
        first_var = csp.variables[0]
        domain = csp.domains[first_var]
        
        # Rozdělení domény
        chunk_size = len(domain) // self.n_workers
        if chunk_size == 0:
            chunk_size = 1
        
        tasks = []
        for i, worker in enumerate(self.workers):
            start_idx = i * chunk_size
            end_idx = start_idx + chunk_size if i < self.n_workers - 1 else len(domain)
            
            if start_idx < len(domain):
                # Vytvoření podproblému
                sub_domain = domain[start_idx:end_idx]
                
                for value in sub_domain:
                    initial_assignment = {first_var: value}
                    task = worker.solve_subproblem.remote(
                        csp, initial_assignment, csp.variables
                    )
                    tasks.append(task)
        
        # Čekání na výsledky
        ready_tasks, remaining_tasks = ray.wait(tasks, num_returns=1)
        
        while ready_tasks:
            result = ray.get(ready_tasks[0])
            if result is not None:
                # Zrušení zbývajících úloh
                for task in remaining_tasks:
                    ray.cancel(task)
                return result
            
            if remaining_tasks:
                ready_tasks, remaining_tasks = ray.wait(
                    remaining_tasks, num_returns=1
                )
            else:
                break
        
        return None
    
    def _solve_variable_split(self, csp):
        """Rozdělí problém podle proměnných"""
        # Rozdělení proměnných do skupin
        var_groups = [[] for _ in range(self.n_workers)]
        for i, var in enumerate(csp.variables):
            var_groups[i % self.n_workers].append(var)
        
        # TODO: Implementovat variable splitting strategii
        pass
    
    def get_statistics(self):
        """Získá statistiky ze všech workerů"""
        stats_futures = [worker.get_stats.remote() for worker in self.workers]
        stats = ray.get(stats_futures)
        
        total_nodes = sum(s['nodes_explored'] for s in stats)
        return {
            'total_nodes_explored': total_nodes,
            'worker_stats': stats
        }

# Portfolio solver - různé strategie paralelně
class PortfolioCSPSolver:
    def __init__(self, strategies):
        self.strategies = strategies
    
    def solve(self, csp, timeout=60):
        """Spustí různé strategie paralelně"""
        with ProcessPoolExecutor(max_workers=len(self.strategies)) as executor:
            # Spuštění všech strategií
            future_to_strategy = {}
            for strategy in self.strategies:
                future = executor.submit(self._run_strategy, csp, strategy)
                future_to_strategy[future] = strategy
            
            # Čekání na první řešení
            for future in as_completed(future_to_strategy, timeout=timeout):
                try:
                    result = future.result()
                    if result is not None:
                        # Zrušení ostatních
                        for f in future_to_strategy:
                            if f != future:
                                f.cancel()
                        
                        return result, future_to_strategy[future]
                except Exception as e:
                    print(f"Strategy {future_to_strategy[future]} failed: {e}")
        
        return None, None
    
    def _run_strategy(self, csp, strategy):
        """Spustí jednu strategii"""
        # Implementace různých strategií
        if strategy == 'backtrack_mrv':
            solver = BacktrackingSolver(csp)
            return solver.solve()
        elif strategy == 'ac3_backtrack':
            # AC3 preprocessing
            propagator = AdvancedConstraintPropagation(csp)
            consistent, reduced_domains = propagator.ac3()
            if not consistent:
                return None
            # Backtracking s redukovanými doménami
            # TODO: Implementovat
            pass
        # Další strategie...
        return None

# Demonstrace paralelního řešení
print("\n🚀 Demonstrace paralelního CSP solving:")

# Vytvoření většího problému pro test
test_problem = NQueensProblem(10)

# Sekvenční řešení
print("Sekvenční řešení...")
start_time = time.time()
seq_solver = BacktrackingSolver(test_problem)
seq_solution = seq_solver.solve()
seq_time = time.time() - start_time
print(f"Čas: {seq_time:.3f}s, Uzlů: {seq_solver.nodes_explored}")

# Paralelní řešení
print("\nParalelní řešení (4 workery)...")
start_time = time.time()
par_solver = ParallelCSPSolver(n_workers=4)
par_solution = par_solver.solve(test_problem)
par_time = time.time() - start_time

if par_solution:
    stats = par_solver.get_statistics()
    print(f"Čas: {par_time:.3f}s")
    print(f"Celkem prozkoumaných uzlů: {stats['total_nodes_explored']}")
    print(f"Zrychlení: {seq_time/par_time:.2f}x")
    
    # Vizualizace práce workerů
    plt.figure(figsize=(10, 6))
    worker_ids = [s['worker_id'] for s in stats['worker_stats']]
    nodes_explored = [s['nodes_explored'] for s in stats['worker_stats']]
    
    plt.bar(worker_ids, nodes_explored, color='skyblue')
    plt.xlabel('Worker ID')
    plt.ylabel('Prozkoumaných uzlů')
    plt.title('Distribuce práce mezi workery')
    plt.show()

ModuleNotFoundError: No module named 'ray'

## 6. Pokročilý hybridní CSP solver 🎯

Kombinace všech technik do jednoho pokročilého řešiče.

In [7]:
# Pokročilý hybridní CSP solver
class AdvancedHybridCSPSolver:
    def __init__(self, neural_model=None):
        self.neural_model = neural_model
        self.statistics = {
            'preprocessing_time': 0,
            'solving_time': 0,
            'total_nodes': 0,
            'propagations': 0,
            'symmetries_broken': 0
        }
    
    def solve(self, csp, config=None):
        """Hlavní metoda řešení s pokročilými technikami"""
        if config is None:
            config = self._get_default_config()
        
        # 1. Preprocessing
        start = time.time()
        csp = self._preprocess(csp, config)
        self.statistics['preprocessing_time'] = time.time() - start
        
        # 2. Řešení
        start = time.time()
        solution = self._solve_with_strategy(csp, config)
        self.statistics['solving_time'] = time.time() - start
        
        return solution
    
    def _get_default_config(self):
        return {
            'use_ac3': True,
            'use_symmetry_breaking': True,
            'use_global_constraints': True,
            'use_neural_guidance': True,
            'parallel_workers': 4,
            'timeout': 300
        }
    
    def _preprocess(self, csp, config):
        """Preprocessing fáze"""
        
        # 1. Arc consistency
        if config['use_ac3']:
            propagator = AdvancedConstraintPropagation(csp)
            consistent, reduced_domains = propagator.ac3()
            if not consistent:
                return None
            
            # Aktualizace domén
            for var in csp.variables:
                csp.domains[var] = reduced_domains[var]
            
            self.statistics['propagations'] = propagator.revision_count
        
        # 2. Symmetry breaking
        if config['use_symmetry_breaking']:
            detector = SymmetryDetector(csp)
            breaking_constraints = detector.add_symmetry_breaking_constraints()
            for constraint in breaking_constraints:
                csp.add_constraint(constraint)
            
            self.statistics['symmetries_broken'] = len(breaking_constraints)
        
        # 3. Global constraint propagation
        if config['use_global_constraints'] and hasattr(csp, 'global_constraints'):
            consistent, reduced_domains = csp.propagate_global_constraints(
                csp.domains
            )
            if not consistent:
                return None
        
        return csp
    
    def _solve_with_strategy(self, csp, config):
        """Výběr a spuštění řešící strategie"""
        if csp is None:
            return None
        
        # Výběr strategie na základě charakteristik problému
        strategy = self._select_strategy(csp)
        
        if strategy == 'parallel' and config['parallel_workers'] > 1:
            solver = ParallelCSPSolver(n_workers=config['parallel_workers'])
            solution = solver.solve(csp)
            stats = solver.get_statistics()
            self.statistics['total_nodes'] = stats['total_nodes_explored']
        
        elif strategy == 'neural' and self.neural_model and config['use_neural_guidance']:
            solution = self._neural_guided_search(csp)
        
        else:
            # Fallback na klasický backtracking
            solver = BacktrackingSolver(csp)
            solution = solver.solve()
            self.statistics['total_nodes'] = solver.nodes_explored
        
        return solution
    
    def _select_strategy(self, csp):
        """Vybere nejlepší strategii na základě charakteristik problému"""
        n_vars = len(csp.variables)
        n_constraints = len(csp.constraints)
        avg_domain_size = np.mean([len(d) for d in csp.domains.values()])
        
        # Jednoduchá heuristika
        if n_vars > 20 and avg_domain_size > 10:
            return 'parallel'
        elif n_constraints / (n_vars * (n_vars - 1) / 2) > 0.3:
            return 'neural'
        else:
            return 'backtrack'
    
    def _neural_guided_search(self, csp):
        """Prohledávání vedené neuronovou sítí"""
        # TODO: Implementovat neural-guided search
        pass
    
    def get_statistics(self):
        """Vrátí detailní statistiky řešení"""
        return self.statistics
    
    def visualize_solving_process(self):
        """Vizualizuje proces řešení"""
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # 1. Timeline
        ax = axes[0, 0]
        phases = ['Preprocessing', 'Solving']
        times = [self.statistics['preprocessing_time'], 
                self.statistics['solving_time']]
        ax.barh(phases, times, color=['lightblue', 'lightgreen'])
        ax.set_xlabel('Čas (s)')
        ax.set_title('Časová náročnost fází')
        
        # 2. Statistiky
        ax = axes[0, 1]
        stats_text = f"""
        Prozkoumaných uzlů: {self.statistics['total_nodes']}
        Propagací: {self.statistics['propagations']}
        Eliminovaných symetrií: {self.statistics['symmetries_broken']}
        Celkový čas: {sum(times):.3f}s
        """
        ax.text(0.1, 0.5, stats_text, fontsize=12, va='center')
        ax.axis('off')
        ax.set_title('Statistiky řešení')
        
        # 3. Efektivita technik
        ax = axes[1, 0]
        techniques = ['AC-3', 'Symmetry\nBreaking', 'Global\nConstraints']
        effectiveness = [
            self.statistics['propagations'] / 100,  # Normalizováno
            self.statistics['symmetries_broken'],
            5  # Placeholder
        ]
        ax.bar(techniques, effectiveness, color=['red', 'green', 'blue'])
        ax.set_ylabel('Efektivita')
        ax.set_title('Přínos jednotlivých technik')
        
        # 4. Složitost problému
        ax = axes[1, 1]
        # Placeholder pro složitost
        ax.pie([30, 25, 20, 15, 10], 
               labels=['Constraints', 'Variables', 'Domains', 'Symmetries', 'Other'],
               autopct='%1.1f%%')
        ax.set_title('Rozložení složitosti')
        
        plt.tight_layout()
        plt.show()

# Test pokročilého solveru
print("\n🎯 Test pokročilého hybridního CSP solveru:")

# Vytvoření komplexního problému
complex_problem = create_magic_square_problem(4)
print(f"Magic Square 4x4 (magický součet = 34)")
print(f"Proměnných: {len(complex_problem.variables)}")
print(f"Velikost stavového prostoru: 16! ≈ 2.1×10¹³")

# Řešení
advanced_solver = AdvancedHybridCSPSolver()
solution = advanced_solver.solve(complex_problem)

if solution:
    print("\n✅ Řešení nalezeno!")
    
    # Zobrazení řešení jako matice
    matrix = np.zeros((4, 4), dtype=int)
    for i in range(4):
        for j in range(4):
            matrix[i, j] = solution[f'cell_{i}_{j}']
    
    print("\nMagický čtverec:")
    print(matrix)
    
    # Kontrola součtů
    print("\nKontrola součtů:")
    print(f"Řádky: {[sum(matrix[i, :]) for i in range(4)]}")
    print(f"Sloupce: {[sum(matrix[:, j]) for j in range(4)]}")
    print(f"Diagonály: {[sum(matrix.diagonal()), sum(np.fliplr(matrix).diagonal())]}")
else:
    print("❌ Řešení nenalezeno")

# Vizualizace procesu
advanced_solver.visualize_solving_process()


🎯 Test pokročilého hybridního CSP solveru:


NameError: name 'create_magic_square_problem' is not defined

## 7. Interaktivní pokročilý CSP Explorer 🎮

Gradio aplikace pro exploraci pokročilých CSP technik.

In [8]:
# Pokročilý interaktivní CSP Explorer
class AdvancedCSPExplorer:
    def __init__(self):
        self.current_problem = None
        self.solver = None
        self.propagation_history = []
    
    def create_custom_problem(self, problem_definition):
        """Vytvoří CSP z textové definice"""
        try:
            # Parse definice problému
            lines = problem_definition.strip().split('\n')
            problem = GlobalCSPProblem()
            
            for line in lines:
                if line.startswith('VAR'):
                    # VAR name domain_start domain_end
                    parts = line.split()
                    name = parts[1]
                    domain = list(range(int(parts[2]), int(parts[3]) + 1))
                    problem.add_variable(name, domain)
                
                elif line.startswith('CONSTRAINT'):
                    # CONSTRAINT type var1 var2 ...
                    parts = line.split()
                    constraint_type = parts[1]
                    vars = parts[2:]
                    
                    if constraint_type == 'ALLDIFF':
                        problem.add_global_constraint(AllDifferentGlobal(vars))
                    elif constraint_type == 'SUM':
                        target = int(vars[-1])
                        problem.add_global_constraint(
                            SumConstraint(vars[:-1], target)
                        )
            
            self.current_problem = problem
            return "Problém úspěšně vytvořen!", self._visualize_constraint_graph()
        
        except Exception as e:
            return f"Chyba: {e}", None
    
    def _visualize_constraint_graph(self):
        """Vizualizuje constraint graf"""
        if not self.current_problem:
            return None
        
        G = nx.Graph()
        
        # Přidání uzlů (proměnných)
        for var in self.current_problem.variables:
            G.add_node(var, node_type='variable')
        
        # Přidání uzlů pro omezení
        constraint_id = 0
        for constraint in self.current_problem.constraints + \
                        getattr(self.current_problem, 'global_constraints', []):
            c_node = f'C{constraint_id}'
            G.add_node(c_node, node_type='constraint')
            
            # Hrany k proměnným
            for var in constraint.variables:
                G.add_edge(c_node, var)
            
            constraint_id += 1
        
        # Vizualizace
        plt.figure(figsize=(10, 8))
        pos = nx.spring_layout(G, k=2, iterations=50)
        
        # Barvy uzlů
        node_colors = []
        for node in G.nodes():
            if G.nodes[node]['node_type'] == 'variable':
                node_colors.append('lightblue')
            else:
                node_colors.append('lightcoral')
        
        nx.draw(G, pos, node_color=node_colors, node_size=1000,
                with_labels=True, font_size=10)
        plt.title('Constraint Graf')
        plt.savefig('constraint_graph.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        return 'constraint_graph.png'
    
    def run_propagation(self, propagation_type):
        """Spustí vybranou propagaci"""
        if not self.current_problem:
            return None, "Nejprve vytvořte problém!"
        
        propagator = AdvancedConstraintPropagation(self.current_problem)
        initial_domains = {var: list(self.current_problem.domains[var]) 
                          for var in self.current_problem.variables}
        
        if propagation_type == "AC-3":
            consistent, reduced = propagator.ac3(initial_domains.copy())
        elif propagation_type == "AC-4":
            consistent, reduced = propagator.ac4()
        elif propagation_type == "Path Consistency":
            consistent, reduced = propagator.path_consistency()
        else:
            return None, "Neznámý typ propagace"
        
        # Vizualizace redukce domén
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        vars_list = list(self.current_problem.variables)
        n_vars = len(vars_list)
        x = np.arange(n_vars)
        
        # Původní velikosti domén
        original_sizes = [len(initial_domains[var]) for var in vars_list]
        reduced_sizes = [len(reduced[var]) for var in vars_list]
        
        width = 0.35
        ax1.bar(x - width/2, original_sizes, width, label='Původní', color='lightblue')
        ax1.bar(x + width/2, reduced_sizes, width, label='Po propagaci', color='orange')
        ax1.set_xlabel('Proměnné')
        ax1.set_ylabel('Velikost domény')
        ax1.set_title(f'{propagation_type} - Redukce domén')
        ax1.set_xticks(x)
        ax1.set_xticklabels(vars_list, rotation=45)
        ax1.legend()
        
        # Statistiky
        total_reduction = sum(original_sizes) - sum(reduced_sizes)
        reduction_percent = (total_reduction / sum(original_sizes)) * 100
        
        stats_text = f"""
        Konzistentní: {'Ano' if consistent else 'Ne'}
        Celková redukce: {total_reduction} hodnot
        Procento redukce: {reduction_percent:.1f}%
        Počet revizí: {propagator.revision_count}
        """
        ax2.text(0.1, 0.5, stats_text, fontsize=14, va='center')
        ax2.axis('off')
        
        plt.tight_layout()
        plt.savefig('propagation_result.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        return 'propagation_result.png', stats_text
    
    def detect_symmetries(self):
        """Detekuje symetrie v problému"""
        if not self.current_problem:
            return None, "Nejprve vytvořte problém!"
        
        detector = SymmetryDetector(self.current_problem)
        var_syms = detector.detect_variable_symmetries()
        val_syms = detector.detect_value_symmetries()
        
        # Vizualizace symetrií
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Graf symetrií proměnných
        if var_syms:
            G = nx.Graph()
            G.add_nodes_from(self.current_problem.variables)
            G.add_edges_from(var_syms)
            
            pos = nx.spring_layout(G)
            nx.draw(G, pos, ax=ax1, node_color='lightblue', 
                   node_size=1000, with_labels=True)
            ax1.set_title('Symetrie proměnných')
        else:
            ax1.text(0.5, 0.5, 'Žádné symetrie proměnných', 
                    ha='center', va='center')
            ax1.set_xlim(0, 1)
            ax1.set_ylim(0, 1)
        
        # Tabulka symetrií hodnot
        if val_syms:
            sym_text = "Symetrie hodnot:\n\n"
            for var, syms in val_syms.items():
                if syms:
                    sym_text += f"{var}: {syms}\n"
            ax2.text(0.1, 0.5, sym_text, fontsize=12, va='center')
        else:
            ax2.text(0.5, 0.5, 'Žádné symetrie hodnot', 
                    ha='center', va='center')
        
        ax2.set_xlim(0, 1)
        ax2.set_ylim(0, 1)
        ax2.axis('off')
        
        plt.tight_layout()
        plt.savefig('symmetries.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        summary = f"""
        Detekované symetrie:
        - Symetrické páry proměnných: {len(var_syms)}
        - Proměnné se symetrickými hodnotami: {len([v for v in val_syms if val_syms[v]])}
        """
        
        return 'symmetries.png', summary

# Vytvoření Gradio rozhraní
explorer = AdvancedCSPExplorer()

with gr.Blocks(title="Advanced CSP Explorer") as demo:
    gr.Markdown("# 🔗 Pokročilý CSP Explorer - Propagace a optimalizace")
    
    with gr.Tab("Definice problému"):
        gr.Markdown(
            """
            ### Formát definice:
            ```
            VAR jméno_proměnné min_hodnota max_hodnota
            CONSTRAINT TYP proměnná1 proměnná2 ... [parametry]
            ```
            
            Typy omezení: ALLDIFF, SUM (s cílovým součtem)
            """
        )
        
        problem_input = gr.Textbox(
            label="Definice problému",
            lines=10,
            value="""VAR A 1 4
VAR B 1 4
VAR C 1 4
VAR D 1 4
CONSTRAINT ALLDIFF A B C D
CONSTRAINT SUM A B 5"""
        )
        
        create_btn = gr.Button("Vytvořit problém", variant="primary")
        problem_status = gr.Textbox(label="Status")
        constraint_graph = gr.Image(label="Constraint graf")
        
        create_btn.click(
            fn=explorer.create_custom_problem,
            inputs=problem_input,
            outputs=[problem_status, constraint_graph]
        )
    
    with gr.Tab("Propagace omezení"):
        propagation_type = gr.Radio(
            choices=["AC-3", "AC-4", "Path Consistency"],
            value="AC-3",
            label="Typ propagace"
        )
        
        propagate_btn = gr.Button("Spustit propagaci", variant="primary")
        propagation_result = gr.Image(label="Výsledek propagace")
        propagation_stats = gr.Textbox(label="Statistiky")
        
        propagate_btn.click(
            fn=explorer.run_propagation,
            inputs=propagation_type,
            outputs=[propagation_result, propagation_stats]
        )
    
    with gr.Tab("Detekce symetrií"):
        detect_btn = gr.Button("Detekovat symetrie", variant="primary")
        symmetry_viz = gr.Image(label="Vizualizace symetrií")
        symmetry_summary = gr.Textbox(label="Shrnutí")
        
        detect_btn.click(
            fn=explorer.detect_symmetries,
            inputs=[],
            outputs=[symmetry_viz, symmetry_summary]
        )
    
    gr.Markdown(
        """
        ### 🚀 Pokročilé techniky:
        - **AC-3/AC-4**: Arc consistency algoritmy
        - **Path Consistency**: Kontrola konzistence cest
        - **Symmetry Detection**: Automatická detekce symetrií
        - **Global Constraints**: Efektivní propagace globálních omezení
        """
    )

# Spuštění aplikace
print("🚀 Spouštím pokročilý CSP Explorer...")
demo.launch(share=True)

NameError: name 'gr' is not defined

## 8. Praktická cvičení 📝

In [9]:
# Cvičení 1: Implementujte GAC (Generalized Arc Consistency)
class GAC:
    """
    TODO: Implementujte GAC pro n-ární omezení
    - Rozšíření AC-3 pro omezení s více než 2 proměnnými
    - Použijte tabulkovou reprezentaci omezení
    """
    def __init__(self):
        pass
    
    def enforce_gac(self, constraint, domains):
        # Váš kód zde
        pass

# Cvičení 2: Implementujte Cumulative constraint
class CumulativeConstraint(GlobalConstraint):
    """
    TODO: Implementujte kumulativní omezení pro scheduling
    - Úlohy s start time, duration, resource requirement
    - Maximální kapacita resources
    """
    def __init__(self, tasks, capacity):
        # Váš kód zde
        pass
    
    def propagate(self, domains):
        # Váš kód zde
        pass

# Cvičení 3: Neuronová síť pro symmetry breaking
class NeuralSymmetryBreaker(nn.Module):
    """
    TODO: Vytvořte GNN pro automatické generování
    symmetry breaking constraints
    """
    def __init__(self):
        super(NeuralSymmetryBreaker, self).__init__()
        # Váš kód zde
        pass
    
    def forward(self, constraint_graph):
        # Váš kód zde
        pass

# Cvičení 4: Implementujte SBDS (Symmetry Breaking During Search)
def sbds_backtrack(csp, assignment, symmetries):
    """
    TODO: Implementujte SBDS - dynamické eliminování symetrií
    během prohledávání
    """
    # Váš kód zde
    pass

print("📚 Cvičení připravena!")
print("\nTipy:")
print("- Pro GAC: Použijte podpory podobně jako AC-4")
print("- Pro Cumulative: Time-table reasoning")
print("- Pro Neural Symmetry: Graph isomorphism network")
print("- Pro SBDS: Udržujte no-goods během search")

📚 Cvičení připravena!

Tipy:
- Pro GAC: Použijte podpory podobně jako AC-4
- Pro Cumulative: Time-table reasoning
- Pro Neural Symmetry: Graph isomorphism network
- Pro SBDS: Udržujte no-goods během search


## 9. Shrnutí a klíčové koncepty 🎓

### Co jsme se naučili:

1. **Pokročilé propagační algoritmy**
   - AC-3, AC-4, Path Consistency
   - Časová složitost a efektivita
   - GAC pro n-ární omezení

2. **Globální omezení**
   - AllDifferent s Hall's theorem
   - Sum constraint s bounds reasoning
   - Cumulative pro scheduling

3. **Symmetry breaking**
   - Detekce symetrií
   - Statické vs dynamické breaking
   - Lexikografické uspořádání

4. **Paralelizace**
   - Domain splitting
   - Portfolio approaches
   - Work stealing

5. **Neuronové přístupy**
   - Učení heuristik propagace
   - Transformer pro constraint learning
   - Hybridní řešení

### Klíčové techniky:
- **Singleton consistency**: Silnější než AC
- **Bounds consistency**: Pro numerické domény
- **Table constraints**: Explicitní reprezentace
- **Incremental propagation**: Efektivní aktualizace

### Další kroky:
- V další hodině: **Úvod do hraní her v AI**
- Začneme s teorií her a adversariálním hledáním

In [10]:
# Závěrečné shrnutí
print("🎉 Gratulujeme! Dokončili jste hodinu o pokročilých CSP technikách!")
print("\n📊 Vaše pokroky:")
print("✅ Implementace pokročilých propagačních algoritmů")
print("✅ Práce s globálními omezeními")
print("✅ Detekce a eliminace symetrií")
print("✅ Paralelní CSP solving")
print("✅ Vytvoření hybridního řešiče")

# Porovnání technik
print("\n📋 Efektivita propagačních technik:")
techniques_comparison = {
    'Technika': ['AC-3', 'AC-4', 'PC-2', 'GAC'],
    'Časová složitost': ['O(ed³)', 'O(ed²)', 'O(n³d³)', 'O(er^d)'],
    'Síla': ['Arc', 'Arc', 'Path', 'GAC'],
    'Použití': ['Binární', 'Binární', 'Binární', 'N-ární']
}

import pandas as pd
df = pd.DataFrame(techniques_comparison)
print(df.to_string(index=False))
print("\ne = počet omezení, d = velikost domény")
print("n = počet proměnných, r = arita omezení")

print("\n🚀 Připraveni na teorii her v další hodině!")

🎉 Gratulujeme! Dokončili jste hodinu o pokročilých CSP technikách!

📊 Vaše pokroky:
✅ Implementace pokročilých propagačních algoritmů
✅ Práce s globálními omezeními
✅ Detekce a eliminace symetrií
✅ Paralelní CSP solving
✅ Vytvoření hybridního řešiče

📋 Efektivita propagačních technik:
Technika Časová složitost Síla Použití
    AC-3           O(ed³)  Arc Binární
    AC-4           O(ed²)  Arc Binární
    PC-2          O(n³d³) Path Binární
     GAC          O(er^d)  GAC  N-ární

e = počet omezení, d = velikost domény
n = počet proměnných, r = arita omezení

🚀 Připraveni na teorii her v další hodině!


# Hodina 15 — Praktické procvičení heuristik a A* (ELI10)

Dnešní hodina je krátké shrnutí a praxe: vezmeme A* z Hodiny 14 a rozšíříme ho o váhy a vizualizaci. Cílem je pochopit, jak heuristika ovlivňuje rychlost hledání a proč musí být admisibilní, pokud chceme být jistí, že najdeme nejkratší cestu.

Níže je jednoduchý projektový scaffolding, který studenti doplní a spustí.

In [ ]:
import heapq
import matplotlib.pyplot as plt
import numpy as np

# Jednoduchý A* pro vážený grid: grid hodnoty >0 udávají náklad pohybu (1 = normální)

def heuristic(a, b):
    # Manhattan
    return abs(a[0]-b[0]) + abs(a[1]-b[1])


def astar_weighted(grid, start, goal):
    rows, cols = grid.shape
    open_set = []
    heapq.heappush(open_set, (0 + heuristic(start, goal), 0, start, None))
    came_from = {}
    g_score = {start: 0}

    while open_set:
        f, g, current, parent = heapq.heappop(open_set)
        if current in came_from:
            continue
        came_from[current] = parent
        if current == goal:
            path = []
            node = current
            while node is not None:
                path.append(node)
                node = came_from[node]
            return list(reversed(path)), g

        r, c = current
        for dr, dc in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < rows and 0 <= nc < cols and grid[nr,nc] < 9999:
                neighbor = (nr,nc)
                tentative_g = g + grid[nr,nc]
                if neighbor in g_score and tentative_g >= g_score[neighbor]:
                    continue
                g_score[neighbor] = tentative_g
                heapq.heappush(open_set, (tentative_g + heuristic(neighbor, goal), tentative_g, neighbor, current))
    return None, float('inf')

# Scaffolding: vytvoř vážený grid a vizualizuj
np.random.seed(0)
rows, cols = 20, 30
base = np.ones((rows, cols))
# náhodné "těžké" buňky
for _ in range(120):
    r = np.random.randint(rows)
    c = np.random.randint(cols)
    base[r,c] = np.random.choice([1,3,5])
# nepřekonatelné překážky
for _ in range(80):
    r = np.random.randint(rows)
    c = np.random.randint(cols)
    base[r,c] = 9999

start = (0,0)
goal = (rows-1, cols-1)
path, cost = astar_weighted(base, start, goal)
print("Path length:", 0 if path is None else len(path)-1, "cost:", cost)
assert path is not None, "Neexistuje cesta — změň hustotu překážek"

# vykreslení
img = base.copy()
for r,c in path:
    img[r,c] = -1
plt.figure(figsize=(10,6))
plt.imshow(img, cmap='tab20c')
plt.title(f'A* weighted path, cost={cost:.1f}')
plt.colorbar(label='cost')
plt.show()

Cvičení a rozšíření

1. Zkuste změnit procento nepřekonatelných buněk a pozorujte, kdy A* přestane nacházet cestu.
2. Implementujte variantu, kde se diagonální pohyb povolí (a diagonální krok má náklad sqrt(2)).
3. Porovnejte rychlost a kvalitu řešení mezi heuristikami: Manhattan, Euclidean a nulová heuristika (Dijkstra).

Tip: Uložte výsledky graficky a proveďte několik běhů s různými semeny náhodného generátoru pro porovnání.